In [3]:
import requests
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
url = "https://raw.githubusercontent.com/langchain-ai/langchain/master/docs/docs/modules/state_of_the_union.txt"
res = requests.get(url)
with open("state_of_the_union.txt", "w") as f:
    f.write(res.text)


In [4]:
# Load the data
loader = TextLoader('./state_of_the_union.txt')
documents = loader.load()

# Chunk the data
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = text_splitter.split_documents(documents)

In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from dotenv import load_dotenv, find_dotenv
# Load OpenAI API key from .env file
load_dotenv(find_dotenv())
# Populate vector database
# FAISS does not need a client setup for local use
vectorstore = FAISS.from_documents(
    documents = chunks,
    embedding = OpenAIEmbeddings()
)
# Define vectorstore as retriever to enable semantic search
retriever = vectorstore.as_retriever()